<a href="https://colab.research.google.com/github/AkbarAzad/AkbarAzad.github.io/blob/main/o2o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Objective: Determine if online behaviour influences offline purchasing behaviour**

# 1. Data preparation


*   Aim to have a dataset of unique users and their online and offline (purchasing) behaviour



In [ ]:
# Mount Colab at Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import packages
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import yaml
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Import dataset
#onlineDf = pd.read_csv('/content/drive/My Drive/web_events.csv')
onlineDf = pd.read_csv('/content/drive/My Drive/web_events_item_brand.csv')
#offlineDf = pd.read_csv('/content/drive/My Drive/showroom_sales.csv')
offlineDf = pd.read_csv('/content/drive/My Drive/showroom_sales_item_brand.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Below are functions designed for this analysis to be reproducible

In [ ]:
# Explore datasets
def showDim(df, name):
  rows = df.shape[0]
  columns = df.shape[1]
  columnNames = ', '.join(df.columns)
  return print(f"Number of rows for {name}: {rows}\nNumber of columns for {name}: {columns}\nColumns for {name} are: {columnNames}")

In [ ]:
# Convert fields to datetime
def toDate(df, columns = ["event_date", "event_timestamp"]):
  df = df.copy()
  newColumn1 = columns[0] + "_new"
  newColumn2 = columns[1] + "_new"
  try:
    df[newColumn1] = df[columns[0]].apply(lambda x: datetime.strptime(str(x), "%Y%m%d"))
  except:
    df[newColumn1] = df[columns[0]].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
  try:
    df[newColumn2] = df[columns[1]].apply(lambda x: pd.to_datetime(str(x), unit = "us"))
  except:
    df[newColumn2] = df[columns[1]].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f"))
  return df

In [ ]:
# Check null values
def checkNull(df, threshold = 0.5):
  df = df.copy()
  rows = df.shape[0]
  for i in df.columns:
    nullValues = sum(df[i].isna())
    nullPct = nullValues/rows
    print(f"{nullPct*100}% of {rows} records are null for column {i}")
    if nullPct >= 0.5:
      df.drop(i, axis = 1, inplace = True)
      print(f"Dropped column {i} because percentage of null values exceed threshold of {threshold}")
  return df

In [ ]:
# Remove irrelevant columns
def removeColumns(df, columns = ["Unnamed: 0"]):
  df = df.copy()
  for i in columns:
    df.drop(i, axis = 1, inplace = True)
    print(f"Dropped column {i}...")
  return df

In [ ]:
# Check first and last dates
def checkDates(df, name, column = "event_date_new"):
  return print(f"Date range for {name} from {df[column].min()} to {df[column].max()}")

In [ ]:
# Get dataframe of items
def getItems(df, columns = ["user_id", "items"]):
  dfItems = pd.DataFrame()
  for user in list(set(df[columns[0]])):
    dfUser = df[df[columns[0]]==user]
    dfUserItems = pd.DataFrame()
    for i, j in dfUser.iterrows():
      dfUserItemsJ = pd.DataFrame(yaml.load(j["items"]))
      dfUserItems = pd.concat([dfUserItems, dfUserItemsJ], axis = 0)
    dfUserItems.reset_index(drop = True, inplace = True)
    dfUserItems["user_id"] = user
    dfItems = pd.concat([dfItems, dfUserItems], axis = 0)
    dfItems.reset_index(drop = True, inplace = True)
  return dfItems

In [ ]:
# Get total items purchased
def getTotalItems(dfItems):
  df = dfItems.copy()
  dfGroup = df.groupby(["user_id"])["quantity"].agg(sum)
  dfGroup = pd.DataFrame(dfGroup)
  dfGroup.reset_index(drop = False, inplace = True)
  dfGroup.columns = ["user_id", "offlinePurchases"]
  return dfGroup

In [ ]:
# Get last online visit date
def getFirstOrLastVisit(df, column = "event_date_new", method = "min", type = "offline"):
  df = df.copy()
  if method == "min":
    dfGroup = pd.DataFrame(df.groupby("user_id")["event_date_new"].min())
  elif method == "max":
    dfGroup = pd.DataFrame(df.groupby("user_id")[column].max())
  dfGroup = dfGroup.reset_index(drop = False)
  dfGroup = dfGroup.rename(columns = {column: f"{method}_{type}_visit_date"})
  return dfGroup

In [ ]:
# Get offline users and their online behaviour
def doMerge(offlineDfRem, dfTotalItems, onlineDfRem):
  # Aggregate
  offlineAgg = pd.DataFrame(offlineDfRem.groupby(["user_id"]).size())
  offlineAgg.reset_index(drop = False, inplace = True)
  offlineAgg.columns = ["user_id", "offlineVisits"]

  onlineAgg = pd.DataFrame(onlineDfRem.groupby(["user_id"]).size())
  onlineAgg.reset_index(drop = False, inplace = True)
  onlineAgg.columns = ["user_id", "onlineVisits"]

  dfMerge = pd.merge(offlineAgg, dfTotalItems, how = "left", on = "user_id")
  dfMerge = pd.merge(dfMerge, onlineAgg, how = "left", on = "user_id")

  return dfMerge

In [ ]:
# Extract binary features
def getFeatures(dfMerge):
  df = dfMerge.copy()
  df["onlineVisits"] = df["onlineVisits"].apply(lambda x: x if x > 0 else 0)
  df["hasPurchasedOffline"] = df["offlinePurchases"].apply(lambda x: 1 if x > 0 else 0)
  df["hasVisitedOnline"] = df["onlineVisits"].apply(lambda x: 1 if x > 0 else 0)
  return df

In [ ]:
# Combine users who visited offline and those who did not but yet visited online
def doConcat(dfFeatures, onlineDfRem):
  userList = list(set(dfFeatures["user_id"]))
  df = onlineDfRem.copy()
  df = df[~df["user_id"].isin(userList)]
  df = df[~df["user_id"].isna()]
  dfGroup = pd.DataFrame(df.groupby("user_id").size())
  dfGroup.reset_index(drop = False, inplace = True)
  dfGroup.columns = ["user_id", "onlineVisits"]
  dfGroup["hasVisitedOnline"] = 1
  dfOut = pd.DataFrame()
  for column in list(dfFeatures.columns):
    if column not in list(dfGroup.columns):
      dfOut[column] = 0
    else:
      dfOut[column] = dfGroup[column]
  dfConcat = pd.concat([dfFeatures, dfOut], axis = 0).reset_index(drop = True)
  if len(list(set(dfConcat["user_id"]))) == dfConcat.shape[0]:
    print("Unique records by user_id...")
  else:
    print("Duplicate records by user_id found!")
  return dfConcat

In [ ]:
# Combine visit dates
def getFirstLastVisits(df1, df2, dfConcat):
  df = dfConcat.copy()
  df = pd.merge(df, df1, how = "left", on = "user_id")
  df = pd.merge(df, df2, how = "left", on = "user_id")
  df = df.reset_index(drop = True)
  return df

In [ ]:
showDim(onlineDf, name = "onlineDf")
showDim(offlineDf, name = "offlineDf")

# Observations:
# 1. More records online than offline
# 2. Remove Unnamed: 0 columns
# 3. Might need to convert date fields to datetime format

Number of rows for onlineDf: 775248
Number of columns for onlineDf: 8
Columns for onlineDf are: Unnamed: 0, event_date, event_timestamp, event_name, user_pseudo_id, user_id, session_id, items
Number of rows for offlineDf: 224
Number of columns for offlineDf: 6
Columns for offlineDf are: Unnamed: 0, event_date, event_timestamp, event_name, user_id, items


In [ ]:
# Show data types
print("Data types for onlineDf:\n", onlineDf.dtypes)
print("Data types for offlineDf:\n", offlineDf.dtypes)

Data types for onlineDf:
 Unnamed: 0           int64
event_date           int64
event_timestamp      int64
event_name          object
user_pseudo_id     float64
user_id            float64
session_id           int64
items               object
dtype: object
Data types for offlineDf:
 Unnamed: 0          int64
event_date         object
event_timestamp    object
event_name         object
user_id             int64
items              object
dtype: object


In [ ]:
onlineDfProcessed = toDate(onlineDf)

In [ ]:
offlineDfProcessed = toDate(offlineDf)

In [ ]:
onlineDfProcessedRem = checkNull(onlineDfProcessed)

0.0% of 775248 records are null for column Unnamed: 0
0.0% of 775248 records are null for column event_date
0.0% of 775248 records are null for column event_timestamp
0.0% of 775248 records are null for column event_name
0.0% of 775248 records are null for column user_pseudo_id
42.882793635069035% of 775248 records are null for column user_id
0.0% of 775248 records are null for column session_id
97.36845499762656% of 775248 records are null for column items
Dropped column items because percentage of null values exceed threshold of 0.5
0.0% of 775248 records are null for column event_date_new
0.0% of 775248 records are null for column event_timestamp_new


In [ ]:
offlineDfProcessedRem = checkNull(offlineDfProcessed)

0.0% of 224 records are null for column Unnamed: 0
0.0% of 224 records are null for column event_date
0.0% of 224 records are null for column event_timestamp
0.0% of 224 records are null for column event_name
0.0% of 224 records are null for column user_id
0.0% of 224 records are null for column items
0.0% of 224 records are null for column event_date_new
0.0% of 224 records are null for column event_timestamp_new


In [ ]:
onlineDfRem = removeColumns(onlineDfProcessedRem)

Dropped column Unnamed: 0...


In [ ]:
offlineDfRem = removeColumns(offlineDfProcessedRem)

Dropped column Unnamed: 0...


In [ ]:
checkDates(onlineDfRem, name = "online")

Date range for online from 2022-02-17 00:00:00 to 2022-02-28 00:00:00


In [ ]:
checkDates(offlineDfRem, name = "offline")

Date range for offline from 2022-02-21 00:00:00 to 2022-02-27 00:00:00


In [ ]:
dfItems = getItems(offlineDfRem)
dfItems.head()

,item_id,item_brand,price,quantity,user_id
0,40550133,E,758.00,1.0,136192
1,50440764-MC4001,BD,1237.00,1.0,136192
2,41960013,AX,888.29,1.0,136193
3,50440257-PL4001,BK,844.63,1.0,136193
4,40550076,U,750.68,1.0,136193


In [ ]:
dfTotalItems = getTotalItems(dfItems)
dfTotalItems.head()

,user_id,offlinePurchases
0,60,1.0
1,1488,1.0
2,7269,2.0
3,9899,1.0
4,11391,2.0


In [ ]:
dfMerge = doMerge(offlineDfRem, dfTotalItems, onlineDfRem)
dfMerge.head()

,user_id,offlineVisits,offlinePurchases,onlineVisits
0,60,1,1.0,28.0
1,1488,1,1.0,27.0
2,7269,1,2.0,73.0
3,9899,1,1.0,57.0
4,11391,1,2.0,NaN


In [ ]:
dfFeatures = getFeatures(dfMerge)
dfFeatures.head()

,user_id,offlineVisits,offlinePurchases,onlineVisits,hasPurchasedOffline,hasVisitedOnline
0,60,1,1.0,28.0,1,1
1,1488,1,1.0,27.0,1,1
2,7269,1,2.0,73.0,1,1
3,9899,1,1.0,57.0,1,1
4,11391,1,2.0,0.0,1,0


In [ ]:
dfConcat = doConcat(dfFeatures, onlineDfRem)
dfConcat.head()
# Include days diff between max online visit day and min offline visit day to look at time taken in days

Unique records by user_id...


,user_id,offlineVisits,offlinePurchases,onlineVisits,hasPurchasedOffline,hasVisitedOnline
0,60.0,1,1.0,28.0,1,1
1,1488.0,1,1.0,27.0,1,1
2,7269.0,1,2.0,73.0,1,1
3,9899.0,1,1.0,57.0,1,1
4,11391.0,1,2.0,0.0,1,0


In [ ]:
lastOnlineVisit = getFirstOrLastVisit(onlineDfRem, method = "min", type = "online")
firstOfflineVisit = getFirstOrLastVisit(offlineDfRem, method = "min", type = "offline")
dfConcat = getFirstLastVisits(lastOnlineVisit, firstOfflineVisit, dfConcat)
dfConcat["days_between"] = dfConcat["min_offline_visit_date"] - dfConcat["min_online_visit_date"]
dfConcat["days_between_v2"] = dfConcat["days_between"].apply(lambda x: float(x.days) if x.days >= 0 else None) # Only for users who first visited online on or before they first visited the physical store
dfConcat.head()

,user_id,offlineVisits,offlinePurchases,onlineVisits,hasPurchasedOffline,hasVisitedOnline,min_online_visit_date,min_offline_visit_date,days_between,days_between_v2
0,60.0,1,1.0,28.0,1,1,2022-02-21,2022-02-22,1 days,1.0
1,1488.0,1,1.0,27.0,1,1,2022-02-19,2022-02-23,4 days,4.0
2,7269.0,1,2.0,73.0,1,1,2022-02-17,2022-02-26,9 days,9.0
3,9899.0,1,1.0,57.0,1,1,2022-02-18,2022-02-21,3 days,3.0
4,11391.0,1,2.0,0.0,1,0,NaT,2022-02-27,NaT,NaN


Summary of findings from data preparation:


*   Online visits are captured from 17 to 28 February 2022, over 2 days while offline visits/purchases are captured from 21 to 27 February 2022. What this means is that we are unable to see users who made online visits between 17 and 18 Feb, and made their way down to the physical store on the day itself till before 21 Feb or after 27 Feb 2022
*   Anyone who visited physical store made at least one purchase (visualised in next section)
*   Able to map both datasets (online and offline) by `user_id` and not by `user_session_id`, which means about 42.9% of records in the online dataset will not be mapped to the offline dataset
*   We should add fields like `channel` to design attribution models and determine the most effective channel in terms of reach







## 2. Exploratory data analyses


*   Let's find answers to important questions using aggregations and visuals



Question 1: How many users do we have in this prepared dataset?

In [ ]:
print(f"We have {dfConcat.shape[0]} users")

We have 4030 users


Question 2: How many users made online visits and how many else did not?

In [ ]:
q2 = pd.DataFrame(dfConcat.groupby("hasVisitedOnline").size()).reset_index(drop = False, inplace = False)
q2 = q2.rename(columns = {0: "users"}, inplace = False)
q2["hasVisitedOnline"] = q2["hasVisitedOnline"].apply(lambda x: "Yes" if x == 1 else "No")
fig = px.bar(q2, x = "hasVisitedOnline", y = "users", text_auto = True, width = 720)
fig.update_layout(title = "Number of users who did not visit online vs those who did", bargap = 0.05)
fig.update_traces(marker_line=dict(width=2, color= 'black'))
fig.show()

From the chart above, 98.2% of these users visited the online site. This shows us that possibly this company's does well in engaging with most of its customers online to some extent.

Question 3: How many users visited the physical store?

In [ ]:
q3 = pd.DataFrame(dfConcat.groupby("hasPurchasedOffline").size()).reset_index(drop = False, inplace = False)
q3 = q3.rename(columns = {0: "users"}, inplace = False)
q3["hasPurchasedOffline"] = q3["hasPurchasedOffline"].apply(lambda x: "Yes" if x == 1 else "No")
fig = px.bar(q3, x = "hasPurchasedOffline", y = "users", text_auto = True, width = 720)
fig.update_layout(title = "Number of users who did not visit offline vs those who did", bargap = 0.05)
fig.update_traces(marker_line=dict(width=2, color= 'black'))
fig.show()

Only 5.4% of these users made offline purchases. This further shows that this company's customers perform online shopping to much greater degree compared to offline shopping. I.e. customers are, for a majority of them, are online shoppers.

Question 4: What is the number of users by whether they visited online and if they visited offline?

In [ ]:
q4 = pd.DataFrame(dfConcat.groupby(["hasVisitedOnline", "hasPurchasedOffline"]).size()).reset_index(drop = False)
q4["hasVisitedOnline"] = q4["hasVisitedOnline"].apply(lambda x: "Yes" if x == 1 else "No")
q4["hasPurchasedOffline"] = q4["hasPurchasedOffline"].apply(lambda x: "Yes" if x == 1 else "No")
q4 = q4.rename(columns = {0: "users"})
fig = px.bar(q4, x = "hasVisitedOnline", y = "users", color = "hasPurchasedOffline", text_auto = True, width = 720)
fig.update_layout(title = "Number of users by online visit and offline visit", bargap = 0.05, barmode = 'group', bargroupgap = 0.03)
fig.update_traces(marker_line=dict(width=2, color= 'black'))
fig.show()


Out of those 3,957 users who visited online, 3.6% made offline purchases.

Question 5: What is the distribution of number of online visits among users?

In [ ]:
# Out of those who visited online, what's the distribution of online visits?
q5_avg = dfConcat["onlineVisits"][dfConcat["hasVisitedOnline"] == 1].mean()
fig = px.histogram(dfConcat[dfConcat["hasVisitedOnline"] == 1], x = "onlineVisits")
fig.update_layout(title = "Distribution of number of online visits by user_id", bargap = 0.1)
fig.update_traces(marker_line = dict(width = 2, color = 'black'))
#fig.add_shape(type = "line", line_color = "red", line_width = 3, opacity = 1, line_dash = "dot", x0 =47, x1 = 47, y0 = 0, y1 = 600, xref = "paper", yref = "y")
fig.add_vline(x = q5_avg, line_width = 3, line_dash = "dash", line_color = "red")
fig.add_annotation(text = f"Average number of online visits at {int(round(q5_avg, 0))}", x = q5_avg, y = 600, arrowhead = 1, showarrow = True, bordercolor = "black", bgcolor = "white", opacity = 1)
fig.show()
# About 2/3 of users visited online store more than once

Based on the average of 112 online visits across 12 days (from 17 Feb to 28 Feb), this means a user visits the online store almost 10 times in a day. (*Using average instead of median despite skewed data, for ease of interpretation)

Question 6: How many users, whether they visited online or otherwise, visited the physical store more than once?

In [ ]:
q6 = pd.DataFrame(dfConcat[dfConcat["hasPurchasedOffline"] == 1].groupby(["offlineVisits", "hasVisitedOnline"]).size()).reset_index(drop = False)
q6 = q6.rename(columns = {0: "users"})
q6["hasVisitedOnline"] = q6["hasVisitedOnline"].apply(lambda x: "Yes" if x == 1 else "No")
fig = px.bar(q6, x = "offlineVisits", y = "users", color = "hasVisitedOnline", text_auto = True, width = 1080)
fig.update_layout(title = "Number of offline visits between those who did not visit online and those who did", bargap = 0.1, barmode = 'group', bargroupgap = 0.03)
fig.update_traces(marker_line=dict(width=2, color= 'black'))
fig.show()

In total, we witness that users who shopped offline and visited online are doubled, compared to those who did not visit online.

In [ ]:
# Move to modelling